In [1]:
import pandas as pd
import numpy as np
import spacy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from spacy.cli.download import download

In [2]:
def criar_embeddings(df_tweets: pd.Series):
    df_tweets.dropna(inplace=True)
    try:
        nlp = spacy.load('pt_core_news_lg')
    except (IOError, OSError):
        download('pt_core_news_lg')
        nlp = spacy.load('pt_core_news_lg')
    # desativamos todos os outros pipes que vem com o modelo nlp porque não preicsaremos deles
    with nlp.disable_pipes():
        # transformamos cada texto em um vetor e colocamos em uma array
        print('Fazendo os word embeddings')
        vetores = np.array([nlp(texto).vector for texto in df_tweets])

    return vetores

## Análise de polaridade de sentimentos

In [3]:
# importação dos dados
path_datasets = '../resources/datasets'

df_treinamento = pd.read_csv(
    f'{path_datasets}/Tweets_Mg.csv',
    usecols=['Text', 'Classificacao']
)
df_alvo = pd.read_csv(f'{path_datasets}/tweets_pandemia.csv')

x_train, x_test, y_train, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Classificacao'],
    test_size=0.2,
    random_state=42
)

# processamento dos dados para word embeddings
embeddings = criar_embeddings(x_train['Text'])
embeddings_teste = criar_embeddings(x_test['Text'])

# algoritmos de aprendizado de máquina para serem treinados
svc = LinearSVC(C=100, random_state=0, dual=True, max_iter=10000)
tree = DecisionTreeClassifier(random_state=0)
forest = RandomForestClassifier(random_state=0)

# treinamento dos algoritmos
svc.fit(embeddings, y_train)
tree.fit(embeddings, y_train)
forest.fit(embeddings, y_train)

# previsões
previsoes_svc = svc.predict(embeddings_teste)
previsoes_tree = tree.predict(embeddings_teste)
previsoes_forest = forest.predict(embeddings_teste)

Fazendo os word embeddings
Fazendo os word embeddings


c:\Users\sefaz\Documents\Projetos Python\Analise-de-sentimentos-pandemia-covid19\tweets_venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [4]:
# testar performance
print('Relatório Linear SVC')
print(classification_report(y_test, previsoes_svc))

Relatório Linear SVC
              precision    recall  f1-score   support

    Negativo       0.94      0.97      0.95       476
      Neutro       0.95      0.89      0.92       503
    Positivo       0.95      0.96      0.96       661

    accuracy                           0.95      1640
   macro avg       0.94      0.94      0.94      1640
weighted avg       0.95      0.95      0.94      1640



In [5]:
print('Relatório Decision Tree')
print(classification_report(y_test, previsoes_tree))

Relatório Decision Tree
              precision    recall  f1-score   support

    Negativo       0.89      0.93      0.91       476
      Neutro       0.84      0.81      0.82       503
    Positivo       0.90      0.89      0.89       661

    accuracy                           0.88      1640
   macro avg       0.87      0.88      0.88      1640
weighted avg       0.88      0.88      0.88      1640



In [6]:
print('Relatório Random Forest')
print(classification_report(y_test, previsoes_forest))

Relatório Random Forest
              precision    recall  f1-score   support

    Negativo       0.99      0.94      0.96       476
      Neutro       0.88      0.93      0.90       503
    Positivo       0.95      0.95      0.95       661

    accuracy                           0.94      1640
   macro avg       0.94      0.94      0.94      1640
weighted avg       0.94      0.94      0.94      1640



O modelo LinearSVC obteve o melhor desempenho em f1-score. Iremos utilizá-lo para a análise de sentimentos dos tweets relacionados à pandemia de Covid-19

## Análise de polaridade nos Tweets da pandemia de Covid-19

In [52]:
def fazer_previsoes(df_tweets: pd.DataFrame, modelo):
    nlp = spacy.load('pt_core_news_lg')
    
    textos = list(df_tweets)
    sentimentos = []

    for texto in textos:
        # print('Frase: ' + texto)
        try:
            # tokenizamos a frase, vetorizamos e colocamos em uma array
            vetor = np.array(nlp(texto).vector)
            # colocamos o vetor no formato 1x300, que é o que o svc espera
            vetor = np.reshape(vetor, (1, 300))
        except TypeError:
            texto = str(texto)
            vetor = np.array(nlp(texto).vector)
            vetor = np.reshape(vetor, (1, 300))
        # fazemos a previsão e retornamos a classe
        previsao = modelo.predict(vetor)[0]
        # print("Sentimento: " + previsao + '\n')
        sentimentos.append(previsao)

    # datas = list(df_tweets['Data'])
    dados = {'Texto': textos, 'Sentimento': sentimentos}
    df_saida = pd.DataFrame(dados)
    
    return df_saida

In [55]:
df_previsoes = fazer_previsoes(df_alvo['Texto'], svc)

In [58]:
df_previsoes['Sentimento'].value_counts()

Sentimento
Neutro      9157
Positivo    5779
Negativo    2065
Name: count, dtype: int64